# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,232.600099,-0.899445,1234,231.700654,233.499544,463.401308,466.999087
6,275.535446,-0.428948,1234,275.106499,275.964394,550.212998,551.928788
7,553.713881,-0.591996,1234,553.121885,554.305876,1106.243770,1108.611753
8,77.709232,-0.300071,1234,77.409161,78.009303,154.818321,156.018607
9,570.666690,-0.071695,1234,570.594995,570.738386,1141.189990,1141.476771
10,750.071392,0.840578,1234,750.911969,749.230814,1501.823939,1498.461627
11,989.467375,-0.842079,1234,988.625296,990.309454,1977.250591,1980.618908
12,120.002248,-0.271774,1234,119.730473,120.274022,239.460947,240.548043
13,259.912700,0.002854,1234,259.915553,259.909846,519.831106,519.819692
14,71.804873,0.762087,1234,72.566960,71.042786,145.133921,142.085571


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-231.700654,233.499544
6,-275.106499,275.964394
7,-553.121885,554.305876
8,-77.409161,78.009303
9,-570.594995,570.738386
10,-750.911969,749.230814
11,-988.625296,990.309454
12,-119.730473,120.274022
13,-259.915553,259.909846
14,-72.566960,71.042786


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-231.700654,233.499544
6,-275.106499,275.964394
7,-553.121885,554.305876
8,-77.409161,78.009303
9,-570.594995,570.738386
10,-750.911969,749.230814
11,-988.625296,990.309454
12,-119.730473,120.274022
13,-259.915553,259.909846
14,-72.566960,71.042786


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-231.70065405,  233.49954371],
       [-275.10649876,  275.96439412],
       [-553.12188494,  554.3058765 ],
       [ -77.40916066,   78.00930328],
       [-570.59499516,  570.7383857 ],
       [-750.91196942,  749.23081363],
       [-988.62529573,  990.30945393],
       [-119.73047346,  120.27402157],
       [-259.91555321,  259.90984597],
       [ -72.56696039,   71.04278567]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,461.401308,-466.999087
6,548.212998,-551.928788
7,1104.243770,-1108.611753
8,152.818321,-156.018607
9,1139.189990,-1141.476771
10,1499.823939,-1498.461627
11,1975.250591,-1980.618908
12,237.460947,-240.548043
13,517.831106,-519.819692
14,143.133921,-142.085571


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
